In [1]:
import numpy
from numba import njit

In [2]:
particle_dtype = numpy.dtype({'names':['x','y','z','m','phi'], 
                             'formats':[numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double]})

# Exercise 1

Write a function `create_n_random_particles` that takes the arguments `n` (number of particles), `m` (mass of every particle) and a domain within to generate a random number (as in the class above).
It should create an array with `n` elements and `dtype=particle_dtype` and then return that array.

For each particle, the mass should be initialized to the value of `m` and the potential `phi` initialized to zero.

For the `x` component of a given particle `p`, you might do something like

```python
p['x'] = domain * numpy.random.random()
```

In [8]:
def create_n_random_particles(n, m, domain=1):
    '''
    Creates `n` particles with mass `m` with random coordinates
    between 0 and `domain`
    '''
    parts = numpy.zeros((n), dtype=particle_dtype)
    
    parts['x'] = numpy.random.random(size=n) * domain
    parts['y'] = numpy.random.random(size=n) * domain
    parts['z'] = numpy.random.random(size=n) * domain
    parts['m'] = m
    parts['phi'] = 0.0

    return parts

Test it out!

In [10]:
parts = create_n_random_particles(1000, .001, 1)
parts[:5]

array([ (0.07865253058714916, 0.17845767290893022, 0.2782564508743751, 0.001, 0.0),
       (0.6098656647837719, 0.465900008549502, 0.7708386758735862, 0.001, 0.0),
       (0.5407396799472325, 0.43441139551555785, 0.5205542751741511, 0.001, 0.0),
       (0.6289394790346508, 0.5203392254721185, 0.510620859464995, 0.001, 0.0),
       (0.08541443823778716, 0.12960520559911615, 0.5964363323868767, 0.001, 0.0)], 
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('m', '<f8'), ('phi', '<f8')])

# Exercise 2

Write a JITted function `distance` to calculate the distance between two particles of dtype `particle_dtype`

Here's the `distance` method from the `Particle` class as a reference:

```python
def distance(self, other):
        return ((self.x - other.x)**2 + 
                (self.y - other.y)**2 + 
                (self.z - other.z)**2)**.5
```

In [23]:
#@njit
def distance(part1, part2):
    '''calculate the distance between two particles'''
    
    return ((part1['x'] - part2['x'])**2 + (part1['y'] - part2['y'])**2 + (part1['z'] - part2['z'])**2)**0.5

Try it out!

In [24]:
distance(parts[0], parts[1])

0.77938933701114987

# Exercise 3

Modify the `direct_sum` function (copied below for reference) to instead work a NumPy array of particles.  Loop over each element in the array and calculate its total potential.

```python
def direct_sum(particles):
    """
    Calculate the potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r
```

In [27]:
#@njit
def direct_sum(particles):
    for i, target in enumerate(particles):
        for j in range(particles.shape[0]):
            if i == j:
                continue
            source = particles[j]
            r = distance(target, source)
            target['phi'] += source['m'] / r

In [28]:
direct_sum(parts)

In [29]:
%timeit direct_sum(parts)

1 loop, best of 3: 8.93 s per loop
